<a href="https://colab.research.google.com/github/bijuthottathil/dataengineer/blob/main/DeltaTablePysparkMergeInCollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
!pip -q install findspark


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages io.delta:delta-core_2.12:0.7.0 --conf spark.sql.extensions=io.delta.sql.DeltaSparkSessionExtension --conf spark.sql.catalog.spark_catalog=org.apache.spark.sql.delta.catalog.DeltaCatalog pyspark-shell'
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName('delta_session').getOrCreate()
from delta.tables import *

In [ ]:
source=spark.read.csv("/content/scd1source.csv", header=True)

In [ ]:
source.write.format("delta").partitionBy("id").mode("overwrite").saveAsTable("source")

In [ ]:
deltasource= DeltaTable.forPath(spark,"/content/spark-warehouse/source")

In [ ]:
deltasource.toDF().show()

+---+--------+---------+
| id|    name| location|
+---+--------+---------+
|  3|Vikranth|Bangalore|
|  2|Reswanth|Hyderabad|
|  1|     Ram|  Chennai|
+---+--------+---------+



In [ ]:
dest=spark.read.csv("/content/scd1dest.csv", header=True)

In [ ]:
dest.write.format("delta").partitionBy("id").mode("overwrite").saveAsTable("dest")

In [ ]:
deltadest= DeltaTable.forPath(spark,"/content/spark-warehouse/dest")

In [ ]:
dfdest=spark.sql("select * from dest")
dfdest.show()

+---+--------+---------+
| id|    name| location|
+---+--------+---------+
|  2|Reswanth|Hyderabad|
|  3|Vikranth|Bangalore|
|  4|     Raj|Hyderabad|
|  5|  Prasad|     Pune|
|  1|     Ram|  Chennai|
+---+--------+---------+



In [ ]:
mergesql=spark.sql("MERGE INTO dest  as t " + 
"USING source as  s on t.id=s.id " +
"WHEN MATCHED THEN  UPDATE SET * WHEN NOT MATCHED THEN INSERT * ")
mergesql.show()


++
||
++
++

